In [3]:
import requests

import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id

In [19]:
questions = {1:'Что мы делаем?', 
             2:'Как называется Ваш проект?', 
             3:'Можете ли Вы сформулировать, какой основной продукт Вы делаете?', 
             4:'В какой более крупной системе оно будет использоваться?',
             5:'Кому нужен результат Вашего проекта?',
             6:'Зачем им нужна Ваша система?',
             7:'Какова их роль по отношению к Вашей системе?',
             8:'Что у Вас в проекте есть уникального и необычного, того, чего нет у других?',
             9:'Почему Ваш проект будет интересен людям?',
             10:'Какие сейчас есть альтернативные решения и подходы к проблеме?',
             11:'Как люди решают проблему сейчас?',
             12:'Сколько стоит или сколько занимает по времени решить проблему сейчас?',
             13:'Какие есть проблемы с текущим решением?',
             14:'Вы можете разделить Ваше решение на части?',
             15:'Перечислите список основных частей',
             16:'Какие из частей проекта готовы или Вы можете заказать и купить в готовом виде?',
             17:'Какие умения нужны для выполнения проекта?',
             18:'Вы можете определить, кто в проекте отвечает за какую часть?',
             19:'Вы можете оценить, сколько времени будет занимать изготовление каждой из частей?'}
answers_neg = {1:'Если у вас есть сложности с названием продукта или системы, то она называется по основной функции, которую выполняет (на какую работу мы её нанимаем)',
               2:'Попробуйте почитать про Branding',
               3:'Попробуйте сформулировать, на какую работу пользователь нанимает Ваш продукт',
               4:'Попробуйте определить несколько объектов, с которыми система/продукт взаимодействует при работе, и назовите частью какой системы они все являются',
               5: '''Вам надо найти людей, которые заинтересованы в вашей системе. Они могут быть покупателями или пользователями,
         инвесторами, представителями сообщества и лидерами мнений, регуляторами (теми, кто может запретить использование продукта). Попробуйте определить
         их роль по отношению к проекту и их интерес в вашем проекте. Подробнее про них написано здесь
            http://sewiki.ru/%D0%A1%D1%82%D0%B5%D0%B9%D0%BA%D1%85%D0%BE%D0%BB%D0%B4%D0%B5%D1%80
           ''' ,
               6: 'Попробуйте найти и опросить больше людей, которым нужна ваша система, определить, какие у них есть проблемы и неудобства (что их расстраивает, их боль)',
               7: 'Понял. Тогда следующий вопрос',
               8: '''У вас в продукте должны быть какие-то параметры, которые на порядок лучше конкурентов. Если вы ещё не разработали уникальные решения, 
              вам возможно потребуются какие-то свежие идеи, научные исследования и технические разработки. Постарайтесь сфокусировать их на основных потребностях
              стейкхолдеров, потребителей проекта, это поможет не распыляться и понять, во что надо вложить максимум усилий, времени и финансов.''',
               9: 'Пообщайтесь с пользователями/потребителями, определите, что их больше всего не устраивает в теущей ситуации',
               10: '''Попробуйте разобраться, является ли проблема действительно актуальной. Не обязательно решение будет продуктом конкурентов,
            конкурентом поездки в другой город на самолёте может оказаться видеозвонок через Интернет или найм местного специалиста для 
            решения проблемы там''',
               11: 'Так не бывает. Либо проблема надумана, либо вы не поняли, что является болью потребителя и вам надо пообщаться с людьми ещё',
               12: 'Постарайтесь оценить сколько люди тратят на решение сейчас. Можно, например, взять цену часа их работы и умножить на то время, которое сейчас расходуется на альтернативное решение.',
               13: 'Попытайтесь разобраться, зачем в таком случае нужен Ваш проект',
               14: '''Ложка состоит из держала и черпала, которые называются по своим основным функциям. Если у держала есть устоявшееся 
            название "ручка", которое все используют, можно перейти к нему.''',
               15: 'Понял. Тогда следующий вопрос',
               16: 'Понял. Тогда следующий вопрос',
               17: 'Возможно, вам надо изучить Practices',
               18: 'Почитайте про современное лидерство, роли в проекте, начальник как джокер',
               19: 'Попросите участников команды оценить работу в условных единицах в формате 1, 2, 3, 5, 8, 13, 20, 40, 100. Можно использовать покер планирования https://scrumpoker.online/'}
    

In [16]:
%load_ext autoreload
%autoreload 2

import deeppavlov
import json
import numpy as np
import tensorflow as tf

from itertools import chain
from pathlib import Path

from deeppavlov import build_model, configs

In [17]:
snips_model = build_model(configs.classifiers.rusentiment_elmo, download=True)

2019-04-07 16:05:03.326 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/rusentiment_v4.tar.gz download because of matching hashes
2019-04-07 16:05:03.341 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from C:\Users\Katerina\.deeppavlov\models\classifiers\rusentiment_v4\classes.dict]
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Katerina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Katerina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     C:\Users\Katerina\AppData\Roaming\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     C:\Users\Katerina\AppData\Roaming\nltk_da

Instructions for updating:
Colocations handled automatically by placer.


W0407 16:05:21.876780 12128 deprecation.py:323] From C:\Users\Katerina\Anaconda3\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0407 16:05:29.703983 12128 saver.py:1483] Saver not created because there are no variables in the graph to restore
2019-04-07 16:05:45.685 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 273: [initializing `KerasClassificationModel` from saved]
I0407 16:05:45.685605 12128 keras_classification_model.py:273] [initializing `KerasClassificationModel` from saved]


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0407 16:05:47.997718 12128 deprecation.py:506] From C:\Users\Katerina\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3363: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-04-07 16:05:49.366 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 283: [loading weights from model.h5]
I0407 16:05:49.366776 12128 keras_classification_model.py:283] [loading weights from model.h5]
2019-04-07 16:05:51.62 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 134: Model was successfully initialized!
Model summary:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to 

__________________________________________________________________________________________________


In [20]:
def main():
    session = requests.Session()
    
    vk_session = vk_api.VkApi(token='here should be your token_ID')
    
    vk = vk_session.get_api()

    upload = VkUpload(vk_session)  # Для загрузки изображений
    longpoll = VkLongPoll(vk_session)
    
    
    
    i = 1
    
    for event in longpoll.listen():
        if event.type == VkEventType.MESSAGE_NEW and event.to_me:
            
            if i<=19:
                message_text = questions[i]
            else:
                message_text = 'Больше вопросов нет. Надеюсь, что помог. Пока!'
            
            
            
            print('id{}: "{}"'.format(event.user_id, event.text), end=' ')
            
                     
           
            
            request = event.text
            
            if snips_model([request])==[['neutral']] and i in range(2,21):
                vk.messages.send( 
                    user_id=event.user_id,
                    random_id=get_random_id(),
                    message=answers_neg[i-1]
                )
            vk.messages.send(
                user_id=event.user_id,
                random_id=get_random_id(),
                message=message_text
                )
            
            
            i+=1
            
            
if __name__ == '__main__':
    main()

id197521036: "Привет. Не мог бы ты мне помочь?" id197521036: "Я не очень в этом разбираюсь, но возможно чат-бота" id197521036: "Не знаю, не определилась" id197521036: "Чат-бот, возможно" id197521036: "Не знаю, без понятия" id197521036: "Никому не нужен(" 

KeyboardInterrupt: 